# Workshop, OpenAI ChatGPT Pizza Bot

In this Notebook we are going to use ChatGPT API in combination with prompt engineering practices to implement a Pizza Bot. 

Python code bellow makes use of the following software:
 - [OpenAI ChatGPT](https://platform.openai.com/docs/introduction) - a language model API


This notebook is also available at [Google Colab](https://colab.research.google.com/drive/1paQ83zH40MLLVu96sRfKwbPTrbYZzeWK?usp=sharing)
 

---
### Dependencies Installation

Make sure required dependencies are installed

In [ ]:
!pip install -q -U openai panel

In [ ]:
import openai
import panel as pn

pn.extension()


In [ ]:
# Globals
openai.api_key = "?"
inp = pn.widgets.TextInput(value="Hello", placeholder='Enter input text here...')
panels = []
context = [{'role': 'system', 'content': """
You are PizzaBot, an automated service to collect orders for a pizza restaurant.

You first greet the customer, then collects the order, and then asks if it's a 
pickup or delivery.

You wait to collect the entire order, then summarize it and check for a final 
time if the customer wants to add anything else.

If it's a delivery, you ask for an address. Finally you collect the payment.

Make sure to clarify all options, extras and sizes to uniquely identify the 
item from the menu.

You respond in a short, very conversational friendly style. 

The menu includes 
Rabo Pepperoni Pizza  12.95, 10.00, 7.00 
Rabo Cheese pizza   10.95, 9.25, 6.50 
Rabo Eggplant pizza   11.95, 9.75, 6.75 
Toppings: 
Extra cheese 2.00, 
Mushrooms 1.50 
Veggie bacon 3.50 
Drinks: 
Water 3.00, 2.00, 1.00 
Tea 3.00, 2.00, 1.00 
Coffe 5.00 
"""}]


### Define utility functions

In [ ]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature,  # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]


def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role': 'user', 'content': f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role': 'assistant', 'content': f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600)))

    return pn.Column(*panels)


### Define Input Components for the demo

In [ ]:
button_conversation = pn.widgets.Button(name="Send")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)
dashboard

In [ ]:
messages = context.copy()
messages.append(
    {
        'role': 'system',
        'content': """create a json summary of the previous food order. Itemize the price for each item
         The fields should be 
         1) pizza, include size 
         2) list of toppings 
         3) list of drinks, include size   
         4) list of sides include size  
         5)total price"""
    },
)

response = get_completion_from_messages(messages, temperature=0)
print(response)

In [ ]:
print(context)